In [77]:
import os
import cv2
from tqdm import tqdm
import json
from xml.etree import ElementTree as ET

anno_path = "E:/IIIT-AR-13K_dataset/validation_xml"
output_annotation_file = "E:/IIIT-AR-13K_dataset/val.json"

In [63]:
coco_data = {}
files = os.listdir(anno_path)    #get all files in the folder
cats = []

for i in tqdm(range(len(files))):
    xml_path = anno_path + '/' + files[i]
    xml = ET.parse(xml_path)
    #print(xml['annotations'])
    
    root = xml.getroot()

    names = root.findall('object/name')
    for name in names:
        if name.text not in cats:
            cats.append(name.text)
cats = sorted(cats)                    

list_dict = []
for i in range(len(cats)):
    tmp = {}
    tmp["supercategory"] = "none"
    tmp["id"] = i
    tmp["name"] = cats[i]
    list_dict.append(tmp)

coco_data["categories"] = list_dict
print(coco_data["categories"])           

100%|████████████████████████████████████| 1955/1955 [00:00<00:00, 7057.36it/s]

[{'supercategory': 'none', 'id': 0, 'name': 'figure'}, {'supercategory': 'none', 'id': 1, 'name': 'logo'}, {'supercategory': 'none', 'id': 2, 'name': 'natural_image'}, {'supercategory': 'none', 'id': 3, 'name': 'signature'}, {'supercategory': 'none', 'id': 4, 'name': 'table'}]


In [76]:
coco_data["images"] = []
coco_data["annotations"] = []

ann_id = 0

for i in tqdm(range(len(files))):
    
    xml_path = anno_path + '/' + files[i]
    xml = ET.parse(xml_path)
    
    root = xml.getroot()

    names = root.findall('filename')
    img_name = names[0].text
    
    sizes = root.findall('size')
    w = sizes[0][0].text
    h = sizes[0][1].text
    
    images_tmp = {}
    images_tmp["file_name"] = img_name
    images_tmp["height"] = h
    images_tmp["width"] = w
    images_tmp["id"] = i
    coco_data["images"].append(images_tmp)
    
    objs = root.findall('object')
    for obj in objs:
        
        label = obj.find('name').text
        x1 = int(obj.find('bndbox/xmin').text)
        y1 = int(obj.find('bndbox/ymin').text)
        x2 = int(obj.find('bndbox/xmax').text)
        y2 = int(obj.find('bndbox/ymax').text)
    
        annotations_tmp = {}
        annotations_tmp["id"] = ann_id
        ann_id += 1
        annotations_tmp["image_id"] = i
        annotations_tmp["segmentation"] = []
        annotations_tmp["ignore"] = 0
        annotations_tmp["area"] = (x2-x1)*(y2-y1)
        annotations_tmp["iscrowd"] = 0
        annotations_tmp["bbox"] = [x1, y1, x2-x1, y2-y1]
        annotations_tmp["category_id"] = cats.index(label)

        coco_data["annotations"].append(annotations_tmp)

100%|████████████████████████████████████| 1955/1955 [00:00<00:00, 5430.25it/s]


In [78]:
outfile =  open(output_annotation_file, 'w')
json_str = json.dumps(coco_data, indent=4)
outfile.write(json_str)
outfile.close()